In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx

pd.options.display.max_columns = None
pd_max_colwidth_original = pd.options.display.max_colwidth
# pd.options.display.max_colwidth = None
gpd.options.io_engine = "pyogrio"

In [2]:
directory_location = r'../../Truck_Areas/'
faf_nodes_df = gpd.read_file(r'zip://' + directory_location + 'FAF5_Model_Highway_Network.zip!Networks/Geodatabase Format/FAF5Network.gdb/a0000000a.gdbtable')
print(f'Number of nodes prior to drop duplicates: {len(faf_nodes_df)}')
faf_nodes_df.drop_duplicates(inplace=True)
faf_nodes_df.drop(faf_nodes_df[faf_nodes_df.State.isin(["HI", "AK"])].index, inplace=True)
faf_nodes_df.reset_index(drop=True, inplace=True)
print(f'Number of nodes after to drop duplicates: {len(faf_nodes_df)}')
faf_links_df = gpd.read_file(r'zip://' + directory_location + 'FAF5_Model_Highway_Network.zip!Networks/Geodatabase Format/FAF5Network.gdb/a00000009.gdbtable')
faf_links_df.drop(faf_links_df[faf_links_df.STATE.isin(["HI", "AK"])].index, inplace=True)
faf_links_df.reset_index(drop=True, inplace=True)

Number of nodes prior to drop duplicates: 974788
Number of nodes after to drop duplicates: 348442


In [3]:
faf_nodes_df.tail(2)

,ID,DATA,Entry_or_Exit,Exit_Number,Interchange,Centroid,CentroidID,Facility_Type,Facility_Name,County,State,StateID,StateName,FAFID,StateNameBak,geometry
348440,1949490,46492932,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-68.76264 44.80220)
348441,1949491,46492239,None,None,None,NaN,NaN,None,None,None,None,NaN,None,NaN,None,POINT (-68.76740 44.80308)


In [4]:
faf_links_df.tail(2)

,ID,LENGTH,DIR,DATA,VERSION,Class,Class_Description,Road_Name,Sign_Rte,Rte_Type,Rte_Number,Rte_Qualifier,Country,STATE,STFIPS,County_Name,CTFIPS,Urban_Code,FAFZONE,Status,F_Class,Facility_Type,NHS,STRAHNET,NHFN,Truck,AB_Lanes,BA_Lanes,Speed_Limit,Toll_Type,Toll_Name,Toll_Link,Toll_Link_Name,HPMS_USA_RouteID,HPMS_Begin_Point,HPMS_End_Point,BorderState1,BorderState2,BorderFAF1,BorderFAF2,TRUCKTOLL,BorderLink,AddedBorderTime,AdjustSpeed,AdjustReason,AB_FinalSpeed,BA_FinalSpeed,AB_CombinedSpeed,BA_CombinedSpeed,AB_FreeFlowTime,BA_FreeFlowTime,SHAPE_Length,geometry
484444,1949503,0.168653,0,669923,V2021.05,14.0,Arterial or Major Collector,BROADWAY,ME 15,S,15,None,USA,ME,23,PENOBSCOT,23019,04951,230.0,1.0,3.0,2.0,10.0,NaN,NaN,None,2.0,2.0,25.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,25.0,25.0,25.0,25.0,0.404766,0.404766,0.002789,"MULTILINESTRING ((-68.77395 44.81775, -68.7743..."
484445,1949504,0.207630,1,775835,V2021.05,22.0,Ramp,RAMP,None,None,None,None,USA,ME,23,PENOBSCOT,23019,04951,230.0,1.0,1.0,4.0,12.0,NaN,NaN,None,1.0,NaN,35.0,NaN,None,NaN,None,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,29.5,29.5,29.5,29.5,0.422299,0.422299,0.003698,"MULTILINESTRING ((-68.77395 44.81775, -68.7737..."


In [5]:
print(f'faf_links_df.csr: {faf_links_df.crs}')
print(f'faf_nodes_df.csr: {faf_nodes_df.crs}')
print(f'faf_links_df.shape: {faf_links_df.shape}')
print(f'faf_nodes_df.shape: {faf_nodes_df.shape}')
print(f'len(faf_links_df): {len(faf_links_df)}')
print(f'len(faf_nodes_df): {len(faf_nodes_df)}')
print(f'len(faf_links_df.columns): {len(faf_links_df.columns)}')
print(f'len(faf_nodes_df.columns): {len(faf_nodes_df.columns)}')

faf_links_df.csr: EPSG:4269
faf_nodes_df.csr: EPSG:4269
faf_links_df.shape: (484446, 53)
faf_nodes_df.shape: (348442, 16)
len(faf_links_df): 484446
len(faf_nodes_df): 348442
len(faf_links_df.columns): 53
len(faf_nodes_df.columns): 16


In [6]:
import shapely

In [7]:
node_lat_long_to_idx = {(row.geometry.coords[0][1], row.geometry.coords[0][0]): row.Index for row in faf_nodes_df.itertuples(index=True)}
len(node_lat_long_to_idx), len(faf_nodes_df)

(348442, 348442)

In [8]:
from_node_idx: list[int] = list()
to_node_idx: list[int] = list()
links_with_no_valid_nodes: list[tuple[int, str, str, str, int, int]] = list()
for row in faf_links_df.itertuples(index=True):
    if shapely.get_num_geometries(row.geometry) > 1:
        print(row)
    linestring = shapely.get_geometry(row.geometry, 0)
    one_end_point = (linestring.coords[0][1], linestring.coords[0][0])
    other_end_point = (linestring.coords[-1][1], linestring.coords[-1][0])
    one_end_point_idx = node_lat_long_to_idx[one_end_point] if one_end_point in node_lat_long_to_idx else -1
    other_end_point_idx = node_lat_long_to_idx[other_end_point] if other_end_point in node_lat_long_to_idx else -1
    from_node_idx.append(one_end_point_idx)
    to_node_idx.append(other_end_point_idx)
       
    if one_end_point_idx == -1 or other_end_point_idx == -1:
        links_with_no_valid_nodes.append((row.ID, row.Country, row.STATE, row.Road_Name, one_end_point_idx, other_end_point_idx))
    
links_with_no_valid_nodes
faf_links_df['FROM_NODE_IDX'] = from_node_idx
faf_links_df['TO_NODE_IDX'] = to_node_idx

In [9]:
len(set(from_node_idx) | set(to_node_idx))

346168

In [10]:
nx_highway_graph = nx.DiGraph()

edge_idx_counter = 0
for row in faf_links_df.itertuples():

    origin_node_idx = row.FROM_NODE_IDX
    destination_node_idx = row.TO_NODE_IDX
    if origin_node_idx == -1 or destination_node_idx == -1:
        continue

    origin_node = faf_nodes_df.loc[origin_node_idx]
    destination_node = faf_nodes_df.loc[destination_node_idx]

    # add the nodes with the right attributese to be used by dyntapy
    nx_highway_graph.add_node(
        origin_node_idx, x_coord=origin_node.geometry.coords[0][0], y_coord=origin_node.geometry.coords[0][1])
    nx_highway_graph.add_node(
        destination_node_idx, x_coord=destination_node.geometry.coords[0][0], y_coord=destination_node.geometry.coords[0][1])

    # add the edges with the right attributes to be used by dyntapy
    # ‘from_node_id’, ‘to_node_id’, ‘link_id’, ‘lanes’, ‘capacity’, ‘length’, ‘free_speed’
    length_miles = row.LENGTH
    ab_lanes = 0 if (np.isnan(row.AB_Lanes) or row.AB_Lanes <
                     1) else row.AB_Lanes  # many links have NaN or 0 lanes
    ba_lanes = 0 if (np.isnan(row.BA_Lanes) or row.BA_Lanes < 1) else row.BA_Lanes
    ab_edge = {
        'u_of_edge': origin_node_idx,
        'v_of_edge': destination_node_idx,
        'from_node_id': origin_node_idx,
        'to_node_id': destination_node_idx,
        'lanes': ab_lanes,
        'length': length_miles,
        'free_speed': row.AB_FinalSpeed,  # still not sure of this
        'capacity': ab_lanes*22000
    }
    ba_edge = {
        'u_of_edge': destination_node_idx,
        'v_of_edge': origin_node_idx,
        'from_node_id': destination_node_idx,
        'to_node_id': origin_node_idx,
        'lanes': ba_lanes,
        'length': length_miles,
        'free_speed': row.BA_FinalSpeed,  # still not sure of this
        'capacity': ba_lanes*22000
    }

    if row.DIR == 1:  # A-> B only
        ab_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ab_edge)
        edge_idx_counter += 1

    elif row.DIR == -1:  # B->A only
        ba_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ba_edge)
        edge_idx_counter += 1

    else:
        # add A->B
        ab_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ab_edge)
        edge_idx_counter += 1

        # add B->A
        ba_edge["link_id"] = edge_idx_counter
        nx_highway_graph.add_edge(**ba_edge)
        edge_idx_counter += 1

print(f"edges {edge_idx_counter:,}")

edges 651,072


In [ ]:
print(f"Is graph strongly connected: {nx.is_strongly_connected(nx_highway_graph)}")
print(f"Number of strongly connected components {len([x for x in nx.strongly_connected_components(nx_highway_graph)]):,}")
print(f"Number of weekly connected components {len([x for x in nx.weakly_connected_components(nx_highway_graph)]):,}")

In [ ]:
s_components = [x for x in nx.strongly_connected_components(nx_highway_graph)]
[ for x in s_components if len(x) > 1]

In [ ]:
s_components[10000]

In [ ]:
lon, lat = faf_nodes_df.loc[120178].geometry.coords[0]
lat, lon

In [ ]:
fake_graph = nx.DiGraph([(0, 1), (1, 2), (2, 3)])
nx.is_strongly_connected(fake_graph)

In [ ]:
faf_nodes_df.loc[[892231,892233]]

In [ ]:
x = faf_links_df.loc[faf_links_df.ID.isin([1784578, 1784579, 1391622])]
x

In [ ]:
faf_links_df.DIR.value_counts()

In [ ]:
faf_nodes_df.loc[faf_links_df.loc[443800].FROM_NODE_IDX]

In [ ]:
faf_nodes_df.loc[faf_links_df.loc[443800].TO_NODE_IDX]

In [ ]:
faf_links_df[(faf_links_df.DIR == -1) & (faf_links_df.STATE == "PA")]

In [ ]:
x.explore()

In [ ]:
faf_nodes_df.loc[[889753,889751]]

In [ ]:
faf_links_df.loc[[444875,444881]]